In [1]:
import dask.dataframe as dd
import json

from dask.distributed import Client, progress

DATA_DIR = 'YOUR DATA DIRECTORY HERE'
DATA_DIR_FULL = DATA_DIR + "PATH TO PARQUET FILES"
Client()

Client Scheduler: tcp://127.0.0.1:51889 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


## Setup

In [4]:
df = dd.read_parquet(DATA_DIR_FULL, columns=['script_url', 'symbol', 'location'])
df = df[['script_url', 'symbol', 'location']]
df.head()

,script_url,symbol,location
0,https://mech.iitm.ac.in/meiitm/wp-includes/js/...,window.navigator.userAgent,https://mech.iitm.ac.in/meiitm/
1,https://mech.iitm.ac.in/meiitm/wp-includes/js/...,window.navigator.userAgent,https://mech.iitm.ac.in/meiitm/
2,https://mech.iitm.ac.in/meiitm/wp-admin/js/iri...,window.navigator.userAgent,https://mech.iitm.ac.in/meiitm/
3,https://mech.iitm.ac.in/meiitm/wp-admin/js/iri...,window.navigator.appName,https://mech.iitm.ac.in/meiitm/
4,https://mech.iitm.ac.in/meiitm/wp-includes/js/...,window.navigator.appVersion,https://mech.iitm.ac.in/meiitm/


## Pull out Candidate URLs for RTCPeerConnection.onicecandidate

In [5]:
onicecandidate_candidate_urls_df = df[df.symbol == 'RTCPeerConnection.onicecandidate']
onicecandidate_candidate_urls_df = onicecandidate_candidate_urls_df[['script_url', 'location']].drop_duplicates().persist()
progress(onicecandidate_candidate_urls_df, notebook=False)

In [6]:
onicecandidate_candidate_urls_df = onicecandidate_candidate_urls_df.compute()
onicecandidate_candidate_urls_df.head()

,script_url,location
2722,https://g.alicdn.com/secdev/sufei_data/3.2.2/i...,https://maniform.world.tmall.com/category-1282...
4180,https://static.adsafeprotected.com/sca.17.4.20.js,https://politikus.ru/2017/11/05/
8071,https://static.adsafeprotected.com/sca.17.4.20.js,https://googleads.g.doubleclick.net/pagead/ads...
17736,https://g.alicdn.com/secdev/sufei_data/3.2.2/i...,https://list.tmall.com/search_product.htm?cat=...
23762,https://static.adsafeprotected.com/sca.17.4.20.js,https://googleads.g.doubleclick.net/pagead/ads...


In [7]:
onicecandidate_candidate_urls = set(onicecandidate_candidate_urls_df.itertuples(index=False))

## Pull out Candidate URLs for RTCPeerConnection.createDataChannel

In [8]:
create_data_channel_candidate_urls_df = df[df.symbol == 'RTCPeerConnection.createDataChannel']
create_data_channel_candidate_urls_df = create_data_channel_candidate_urls_df[['script_url', 'location']].drop_duplicates().persist()
progress(create_data_channel_candidate_urls_df, notebook=False)

In [9]:
create_data_channel_candidate_urls_df = create_data_channel_candidate_urls_df.compute()
create_data_channel_candidate_urls_df.head()

,script_url,location
2723,https://g.alicdn.com/secdev/sufei_data/3.2.2/i...,https://maniform.world.tmall.com/category-1282...
17737,https://g.alicdn.com/secdev/sufei_data/3.2.2/i...,https://list.tmall.com/search_product.htm?cat=...
25944,https://pixel.yabidos.com/flimpobj.js?cb=15133...,https://www.hobbyconsolas.com/videojuegos/hori...
11478,http://g.alicdn.com/secdev/sufei_data/3.2.2/in...,http://c.youku.com/abouteg/youtu
21311,https://g.alicdn.com/secdev/sufei_data/3.2.2/i...,https://list.tmall.com//search_product.htm?cat...


In [10]:
create_data_channel_candidate_urls = set(create_data_channel_candidate_urls_df.itertuples(index=False))

## Pull out Candidate URLs for RTCPeerConnection.createOffer

In [11]:
create_offer_candidate_urls_df = df[df.symbol == 'RTCPeerConnection.createOffer']
create_offer_candidate_urls_df = create_offer_candidate_urls_df[['script_url', 'location']].drop_duplicates().persist()
progress(create_offer_candidate_urls_df, notebook=False)

In [12]:
create_offer_candidate_urls_df = create_offer_candidate_urls_df.compute()
create_offer_candidate_urls_df.head()

,script_url,location
2724,https://g.alicdn.com/secdev/sufei_data/3.2.2/i...,https://maniform.world.tmall.com/category-1282...
17738,https://g.alicdn.com/secdev/sufei_data/3.2.2/i...,https://list.tmall.com/search_product.htm?cat=...
25945,https://pixel.yabidos.com/flimpobj.js?cb=15133...,https://www.hobbyconsolas.com/videojuegos/hori...
11479,http://g.alicdn.com/secdev/sufei_data/3.2.2/in...,http://c.youku.com/abouteg/youtu
21312,https://g.alicdn.com/secdev/sufei_data/3.2.2/i...,https://list.tmall.com//search_product.htm?cat...


In [13]:
create_offer_candidate_urls = set(create_offer_candidate_urls_df.itertuples(index=False))

## URLs should call all of: [RTCPeerConnection.onicecandidate, RTCPeerConnection.createDataChannel, RTCPeerConnection.createOffer]

In [14]:
all_candidate_urls = onicecandidate_candidate_urls | create_data_channel_candidate_urls | create_offer_candidate_urls
candidate_urls = onicecandidate_candidate_urls & create_data_channel_candidate_urls & create_offer_candidate_urls

In [15]:
print('# of unique tuples (script_url, location) that call all 3 symbols:', len(candidate_urls))
print('# of unique tuples (script_url, location) that only call 1 or 2 symols:', len(all_candidate_urls - candidate_urls))

# of unique tuples (script_url, location) that call all 3 symbols: 12936
# of unique tuples (script_url, location) that only call 1 or 2 symols: 6614


In [16]:
with open('not_webrtc_multi_symbol_fingerprinting_1.json', 'w') as f:
    f.write(json.dumps(list(set([script_url for script_url, location in (all_candidate_urls - candidate_urls)]))))

Note: conversion from a list to a set to a list above is used in case the same script_url is called from multiple locations

## Drop false positives

In [17]:
false_positives = {
    "nextwebsnetworks.com/in/video",
    "nextwebsnetworks.com/in/remnant",
    "sandbox02.kaxsdc.com/collect/logo.htm",
    "digitalforexnfashionstyles.com/sms4send2.php",
    "www.badlion.net/",
    "ajax.googleapis.com/ajax/libs/jquery/1.8/jquery.min.js"
}

webrtc_fp_urls = set([(script_url, location) for script_url, location in candidate_urls if script_url not in false_positives])

In [18]:
print('# of unique tuples (script_url, location) that were false positives:', len(candidate_urls - webrtc_fp_urls))
print('# of unique tuples (script_url, location) that used WebRTC fingerprinting:', len(webrtc_fp_urls))

# of unique tuples (script_url, location) that were false positives: 0
# of unique tuples (script_url, location) that used WebRTC fingerprinting: 12936


In [19]:
with open('not_webrtc_multi_symbol_fingerprinting_2.json', 'w') as f:
    f.write(json.dumps(list(set([script_url for script_url, location in (candidate_urls - webrtc_fp_urls)]))))

## Save Candidate URLs

In [20]:
with open('webrtc_multi_symbol_fingerprinting.json', 'w') as f:
    f.write(json.dumps(list(set([script_url for script_url, location in webrtc_fp_urls]))))

## Find Locations

In [4]:
with open('webrtc_multi_symbol_fingerprinting.json', 'r') as f:
    webrtc_fp_urls = json.load(f)
    
print(len(webrtc_fp_urls), '== 1313')

1313 == 1313


In [5]:
df = dd.read_parquet(DATA_DIR_FULL, columns=['script_url', 'location'])

In [6]:
df_locs = df[df.script_url.isin(webrtc_fp_urls)]
locs = df_locs.location.unique().persist()
progress(locs, notebook=False)

In [7]:
print('# of locations that call WebRTC fingerprinting scripts:', len(locs))

# of locations that call WebRTC fingerprinting scripts: 15360
